In [559]:
import numpy as np
import random
import itertools

In [560]:
class Game:

    def __init__(self,tricks,players):

        self.tricks = tricks
        self.num_players = len(players)
        self.players = players
        self.start = 0

    def get_cards(self):

        return self.deck
    
    def play(self,preshuffled=[]):

        # Shuffle and distribute the cards
        if preshuffled == []:
            self.deck = list(np.arange(1,self.tricks*self.num_players+1))
            random.shuffle(self.deck)
            self.deck = [self.deck[i:i + self.tricks] for i in range(0, len(self.deck), self.tricks)]
            for i,player in zip(range(self.num_players),self.players):
                player.set_hand(self.deck[i])

        else:
            for i,player in zip(range(self.num_players),self.players):
                player.set_hand(preshuffled[i])

        # Set tricks eating counter
        tricks_eaten = [0] * self.num_players

        # Play rounds
        while self.tricks > 0:
            plays = []
            plays.append(self.players[self.start].play_lowest()) # First player plays card
            for i in range(self.num_players - 1): # Every player plays a card
                if i != self.num_players - 2:
                    plays.append(self.players[(i + 1 + self.start) % self.num_players].respond(max(plays)))
                else:
                    plays.append(self.players[(i + 1 + self.start) % self.num_players].respond(max(plays),last=True))
            self.start = (self.start + plays.index(max(plays))) % self.num_players # Find loser of the round
            tricks_eaten[self.start] += 1
            self.tricks -= 1

        return tricks_eaten
        
class Player:

    def __init__(self,number):

        self.number = number
        self.hand = []

    def set_hand(self,hand):

        self.hand = hand

    def get_hand(self):

        return self.hand
    
    # Used by first player in a round to play the lowest card in their hand
    def play_lowest(self):

        if len(self.hand) == 0:
            return
        lowest = min(self.hand)
        self.hand.remove(min(self.hand))
        return lowest
    
    # Used by non-first players in a round to play a card depending on the highest card played and turn order
    def respond(self,highest,last=False):

        lower_responses = [x for x in self.hand if x < highest] # Generate list of numbers lower than highest card
        if (len(lower_responses) == 0) & (last == False): # Play lowest card in hand
            lowest = min(self.hand)
            self.hand.remove(min(self.hand))
            return lowest
        elif (len(lower_responses) == 0) & (last == True): # Play highest card in hand
            highest = max(self.hand)
            self.hand.remove(max(self.hand))
            return highest
        else: # Play highest card from the list we generated above
            highest_of_low = max(lower_responses)
            self.hand.remove(max(lower_responses))
            return highest_of_low



In [570]:
for integer in range(10):

    my_list = np.arange(0,2*integer)
    combinations = list(itertools.combinations(my_list,integer))
    iteration = []
    for i in combinations:
        iteration.append(([[j for j in i],list(set(my_list)-set([j for j in i]))]))

    player1 = Player(1)
    player2 = Player(2)

    wins = 0
    losses = 0
    draws = 0

    for iter in iteration:
        game = Game(tricks=integer,players=[player1,player2])
        result = game.play(iter)

        if result[0] > result[1]:
            losses += 1
        elif result[0] < result[1]:
            wins += 1
        else:
            draws += 1

    print('Number of Cards per Player:',integer)
    print('Total Games:',len(iteration))
    print('Wins:',round(wins/len(iteration)*100,2))
    print('Draws:',round(draws/len(iteration)*100,2))
    print('Losses:',round(losses/len(iteration)*100,2))

Number of Cards per Player: 0
Total Games: 1
Wins: 0.0
Draws: 100.0
Losses: 0.0
Number of Cards per Player: 1
Total Games: 2
Wins: 50.0
Draws: 0.0
Losses: 50.0
Number of Cards per Player: 2
Total Games: 6
Wins: 16.67
Draws: 50.0
Losses: 33.33
Number of Cards per Player: 3
Total Games: 20
Wins: 40.0
Draws: 0.0
Losses: 60.0
Number of Cards per Player: 4
Total Games: 70
Wins: 25.71
Draws: 27.14
Losses: 47.14
Number of Cards per Player: 5
Total Games: 252
Wins: 38.89
Draws: 0.0
Losses: 61.11
Number of Cards per Player: 6
Total Games: 924
Wins: 30.09
Draws: 18.07
Losses: 51.84
Number of Cards per Player: 7
Total Games: 3432
Wins: 39.1
Draws: 0.0
Losses: 60.9
Number of Cards per Player: 8
Total Games: 12870
Wins: 32.71
Draws: 13.45
Losses: 53.84
Number of Cards per Player: 9
Total Games: 48620
Wins: 39.55
Draws: 0.0
Losses: 60.45
